In [39]:
import matplotlib.pyplot as plt
from classiq import construct_combinatorial_optimization_model
from classiq.applications.combinatorial_optimization import OptimizerConfig, QAOAConfig
import numpy as np
from qiskit import QuantumCircuit as qc
from pyomo.environ import *
from pyomo.opt import SolverFactory
from classiq import show, synthesize
from classiq import execute
import pandas as pd
from classiq.applications.combinatorial_optimization import get_optimization_solution_from_pyo
from classiq import set_execution_preferences
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq import write_qmod
import random as rd
import logging

logging.getLogger('pyomo.core').setLevel(logging.ERROR)

### 1. Generate a pyomo model from a ksat matrix

In [2]:
c_test = [[1,0,0,0],[1,0,1,0],[0,-1,0,0],[0,0,0,-1],[0,0,1,1]]
ctest_2 = [[1,0,0,0,0],[1,0,1,0,0],[0,-1,0,0,0],[0,0,0,-1,0],[0,0,1,1,0],[0,0,0,0,-1],[1,0,0,0,1]]

def model_from_mat(p_mat=c_test):
    cost = 0
    n, m = len(p_mat[0]), len(p_mat)
    model = ConcreteModel()
    model.x = Var(range(n), domain=Binary)
    for i in range(m):
        clause_i = 1
        for j in range(n):
            if(p_mat[i][j] == 1):
                clause_i *= 1-model.x[j]
            if(p_mat[i][j] == -1):
                clause_i *= model.x[j]
        cost += clause_i
    model.obj = Objective(expr=cost, sense=minimize)
    return model

### 2. Solve the model using qaoa

In [60]:
def solve_ksat_qaoa(p_mat=c_test,penalty_energy=2,alpha_cavar=0.9,max_iter=500,backend_name="aer_simulator"):
    model = model_from_mat(p_mat)
    qaoa_config = QAOAConfig(num_layers=1, penalty_energy=2)
    optimizer_config = OptimizerConfig(max_iteration=500, alpha_cvar=0.9)
    qmod = construct_combinatorial_optimization_model(
        pyo_model=model,
        qaoa_config=qaoa_config,
        optimizer_config=optimizer_config,
    )
    backend_preferences = ExecutionPreferences(
        backend_preferences=ClassiqBackendPreferences(backend_name="aer_simulator")
    )
    qmod = set_execution_preferences(qmod, backend_preferences)
    qprog = synthesize(qmod)
    res = execute(qprog).result()
    vqe_result = res[0].value
    solution = get_optimization_solution_from_pyo(
        model, vqe_result=vqe_result, penalty_energy=qaoa_config.penalty_energy
    )
    optimization_result = pd.DataFrame.from_records(solution)
    idx = optimization_result.cost.idxmin()
    sol = optimization_result.solution[idx]
    cost = optimization_result.cost[idx]
    prob = optimization_result.probability[idx]
    table = pd.DataFrame.from_records(solution)
    print(f"QAOA solution: {sol}\n QAOA probability: {prob}\n QAOA cost: {cost}")
    return sol, prob, cost, table

### 3. Generate random matrices encrypting ksat problems

In [ ]:
def generate_ksat_mat(k, m, p=0.5):
    matrix = np.zeros((m, k), dtype=int)
    unique_rows = set()
    negation_rows = set()

    for i in range(m):
        while True:
            row = np.zeros(k, dtype=int)
            for j in range(k):
                if rd.random() < p:  # Probability of including a variable in a clause
                    # Randomly assign either 1 or -1 (true or negated)
                    row[j] = 1 if rd.random() < 0.5 else -1

            row_tuple = tuple(row)
            negated_row_tuple = tuple(-x for x in row)  # Create a tuple of negated values

            # Check that the row is not all zeros, not already included, and not a negation of an included row
            if row_tuple not in unique_rows and np.any(row != 0) and negated_row_tuple not in unique_rows:
                unique_rows.add(row_tuple)
                negation_rows.add(negated_row_tuple)  # Keep track of negations to avoid adding them later
                matrix[i, :] = row
                break

    return matrix.tolist()

### 4. Methods to classically verify ksat solutions

In [ ]:
def verify_solution(matrix, solution):
    num_clauses = len(matrix)
    num_vars = len(matrix[0])
    
    for i in range(num_clauses):
        clause_satisfied = False
        for j in range(num_vars):
            if matrix[i][j] != 0:
                # Determine the truth value of the j-th variable in this clause
                literal_truth = (solution[j] == 1) if matrix[i][j] == 1 else (solution[j] == 0)
                if literal_truth:
                    clause_satisfied = True
                    break  # No need to check more literals in this clause
        if not clause_satisfied:
            return False  # If any clause is not satisfied, return False immediately
    
    return True  # All clauses are satisfied

### 5. Method to get statistics for the solver (adjustable, just replace solve_ksat_qaoa)

In [138]:
def get_statistics(k,m,n,p=0.5):
    # n is the number of iterations
    # p is the probability of including a variable in a clause
    # k is the number of variables per clause
    # m is the number of clauses
    valid_tries = n # Only include matrices that are actually satisfiable
    successes = 0
    for i in range(n):
        print(f"Iteration: {i+1} of {n}")
        ran_mat = generate_ksat_mat(k,m,p)
        print(ran_mat)
        validity = verify_solution(ran_mat,solve_ksat_qaoa(ran_mat)[0])
        if(validity):
            print("Valid solution!\n")
            successes += 1
    print(f"Success rate: {successes/valid_tries}")
    return successes/valid_tries

In [143]:
k_m_vals = range(3,10,1)
sucess_rates = np.zeros(len(k_m_vals))
for i in range(len(k_m_vals)):
    sucess_rates[i] = get_statistics(k_m_vals[i], k_m_vals[i], 20)

Iteration: 1 of 20
[[1, 0, -1], [-1, 0, 0], [-1, 1, 0]]
QAOA solution: [0, 1, 0]
 QAOA probability: 0.47412109375
 QAOA cost: 0.0
Valid solution!

Iteration: 2 of 20
[[0, 0, -1], [0, -1, 0], [1, 0, 0]]
QAOA solution: [1, 0, 0]
 QAOA probability: 0.97900390625
 QAOA cost: 0.0
Valid solution!

Iteration: 3 of 20
[[0, 0, 1], [1, 0, 0], [1, -1, -1]]
QAOA solution: [1, 0, 1]
 QAOA probability: 0.52978515625
 QAOA cost: 0.0
Valid solution!

Iteration: 4 of 20
[[0, -1, -1], [0, 0, -1], [-1, 0, 1]]
QAOA solution: [0, 0, 0]
 QAOA probability: 0.44873046875000006
 QAOA cost: 0.0
Valid solution!

Iteration: 5 of 20
[[1, 0, 1], [0, -1, 0], [0, -1, -1]]
QAOA solution: [1, 0, 0]
 QAOA probability: 0.36083984375
 QAOA cost: 0.0
Valid solution!

Iteration: 6 of 20
[[-1, 0, 0], [-1, 0, 1], [-1, -1, 0]]
QAOA solution: [0, 0, 1]
 QAOA probability: 0.34765624999999994
 QAOA cost: 0.0
Valid solution!

Iteration: 7 of 20
[[0, 0, -1], [1, 0, 1], [0, 1, 1]]
QAOA solution: [1, 1, 0]
 QAOA probability: 0.46875


## Functions not used

In [ ]:
# Solve a ksat problem classically

def solve_ksat_class(matrix=c_test):
    model = ConcreteModel()
    n, m = len(matrix[0]), len(matrix)
    model.x = Var(range(n), domain=Binary)
    model.obj = Objective(expr=0, sense=minimize)
    model.constraints = ConstraintList()

    for i in range(m):
        clause_expr = 0
        for j in range(n):
            if matrix[i][j] == 1:
                clause_expr += model.x[j]  # Variable itself
            elif matrix[i][j] == -1:
                clause_expr += (1 - model.x[j])  # Negation of the variable
        # Add at least one true literal per clause
        model.constraints.add(clause_expr >= 1)
    solver = SolverFactory('glpk')
    result = solver.solve(model, tee=False)
    if (result.solver.status == SolverStatus.ok) and (result.solver.termination_condition == TerminationCondition.optimal):
        # If feasible and optimal, extract the solution
        solarray = [int(model.x[i].value) for i in range(len(model.x))]
    else:
        # If not feasible or not optimal, handle accordingly
        print("No feasible solution found.")
        solarray = None  # or return an empty list or any appropriate indicator

    return solarray, model

# Compare a classical t oa quantum solution

def solve_ksat(matrix=c_test):
    csol, cmodel = solve_ksat_class(matrix)
    qsol, qprob, qcost, table, fullsol = solve_ksat_qaoa(matrix)
    if(csol == qsol):
        print("Both solutions are equal!")
    if(validity):
        print("Quantum solution validated by classical solver!")
    return csol, qsol, qprob, qcost, table